# Download and Format Lawmatics Timeline Activities

This Notebook downloads all timeline activities from Lawmatics, formats them into a tabular format, applies data types, then loads them into a BigQuery datastore table for further processing.

Because there are so many events, this notebook uses temporary tables to store raw retrieval batches and an intermediate table to consolidate all raw retrievals before applying transforms. This notebook is restartable when the download is interrupted.

## Define Date Range for Data Retrieval

This section allows you to define a specific date range for data retrieval. If no dates are provided, the notebook will automatically query the existing `lawmatics_activity_timeline_raw_retrieval` table to find the most recent `attributes.updated_at` timestamp and fetch all new records generated since then (incremental update).

In [1]:
# Define the start and end dates for data retrieval (optional)
# Format: 'YYYY-MM-DD'
# Set to None for incremental updates (retrieves data from the last updated_at in BigQuery)
# START_DATE_STR = None  # Example: '2023-01-01'
START_DATE_STR = '2025-10-21'  # Example: '2023-01-01'
END_DATE_STR = None    # Example: '2023-12-31'

# Note: Due to API limitations, only a single filter (start date) can be applied directly.
# If an END_DATE_STR is provided, it will be ignored by the API call, and you would need to
# apply further filtering in BigQuery if you require an upper bound on the date.

## Reset temp tables if necessary

In [2]:
# from google.cloud import bigquery

# # BigQuery configuration
# project_id = 'www-prod-389819'
# dataset_id = 'mlo_kpi'
# temp_table_prefix = 'tmp_lawmatics_activity_page_'

# bq_client = bigquery.Client(project=project_id)

# # Get all temporary tables
# query = f"""
# SELECT table_id
# FROM `{project_id}.{dataset_id}.__TABLES__`
# WHERE table_id LIKE '{temp_table_prefix}%'
# """

# try:
#     tables_df = bq_client.query(query).to_dataframe()

#     if tables_df.empty:
#         print("No temporary tables found to delete.")
#     else:
#         print(f"Found {len(tables_df)} temporary tables to delete:")

#         deleted_count = 0
#         for table_id in tables_df['table_id']:
#             try:
#                 table_ref = f"{project_id}.{dataset_id}.{table_id}"
#                 bq_client.delete_table(table_ref)
#                 print(f"Deleted: {table_id}")
#                 deleted_count += 1
#             except Exception as e:
#                 print(f"Error deleting {table_id}: {e}")

#         print(f"\nSuccessfully deleted {deleted_count} out of {len(tables_df)} temporary tables.")

# except Exception as e:
#     print(f"Error querying for temporary tables: {e}")

## Install requirements

In [3]:
!pip install google.cloud google-cloud-secret-manager

## Retrieve Lawmatics API Token

In [4]:
# This cell is now redundant as its logic has been moved to cell 41c61070
# The secret retrieval is now integrated directly into the main data retrieval process
# to ensure all necessary variables are defined within the same execution scope.

## Get API JSON Data
Retrieve paginated data from a REST API using a non-expiring OAuth2 access token and transform it into a tabular format for analysis.

## Implement pagination logic

### Subtask:
Handle the API's pagination, making requests for subsequent pages until all data is retrieved.  Directly load each page's data into a temporary BigQuery table (e.g., mlo_kpi.tmp_lawmatics_activity_page_<page_num>) and implement logic to resume data retrieval from the last processed page by checking existing temporary tables.


**Reasoning**:
To implement batch processing, I will adjust the existing API retrieval cell to import necessary BigQuery libraries, define BigQuery table parameters, query BigQuery for the last processed page to enable resume functionality, convert each page's data to a DataFrame, and then load it directly into a temporary BigQuery table.



In [5]:
import requests
import time
import os
import pandas as pd
from google.cloud import bigquery
import pandas_gbq
from datetime import datetime, timedelta, timezone
from google.cloud import secretmanager
from hashlib import sha1

# --- Secret Manager Access Function ---
def access_secret(project_id, secret_id, version_id="latest"):
    """
    Access a secret from Google Cloud Secret Manager

    Args:
        project_id: Your Google Cloud project ID
        secret_id: The ID of the secret
        version_id: The version of the secret (defaults to "latest")

    Returns:
        The secret value as a string
    """
    # Create the Secret Manager client
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version
    response = client.access_secret_version(request={"name": name})

    # Return the decoded secret
    return response.payload.data.decode("UTF-8")

# --- Retrieve Lawmatics API Token ---
project_id_secret = '566451752897'
secret_id = 'mlo-lawmaticsApiKey'

try:
    secret_value = access_secret(project_id_secret, secret_id)
    secret_value_sha1 = sha1(secret_value.encode())
    secret_value_digest = secret_value_sha1.hexdigest()
    print(f"Secret retrieved successfully: (sha1) {secret_value_digest}")
except Exception as e:
    print(f"Error accessing secret: {e}")
    # Exit or raise error if API key is critical

# --- Set up API request ---
api_endpoint = 'https://api.lawmatics.com/v1/activities'
headers = {
    'Authorization': f'Bearer {secret_value}'
}
params = {
    'page': 1,
    'per_page': 100 # Or any other appropriate value based on API documentation
}

# Configuration: Set to None to process all pages, or specify a number to limit pages
MAX_PAGES_TO_PROCESS = None  # Change to a number like 50 to limit processing


# BigQuery configuration
project_id = 'www-prod-389819'
dataset_id = 'mlo_kpi'
temp_table_prefix = 'tmp_lawmatics_activity_page_'
intermediate_table_id = 'lawmatics_activity_timeline_raw_retrieval'

bq_client = bigquery.Client(project=project_id)

# Date formats
# API_DATE_FORMAT is now handled by isoformat directly, so it's removed.
BQ_DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S UTC' # Kept for consistency with potential BQ queries

# Determine the effective start date for API calls
effective_start_date_str = None

if START_DATE_STR:
    # User provided a start date ('YYYY-MM-DD')
    # Treat as start of day in local timezone, then format for API
    try:
        naive_start_dt = datetime.strptime(START_DATE_STR, '%Y-%m-%d')
        # Make it timezone-aware in the system's local timezone
        # datetime.astimezone() with no arguments makes it local timezone aware
        local_start_dt = naive_start_dt.astimezone()
        effective_start_date_str = local_start_dt.isoformat(timespec='milliseconds')
        print(f"Using user-defined start date: {START_DATE_STR}, formatted as {effective_start_date_str} for API.")
    except ValueError as e:
        print(f"Error parsing START_DATE_STR '{START_DATE_STR}': {e}. Falling back to incremental logic.")
        START_DATE_STR = None # Fallback to incremental, so the next 'else' block will run

if not effective_start_date_str: # If START_DATE_STR was None or parsing failed
    # No start date provided by user, perform incremental load
    try:
        query = f"""
        SELECT MAX(SAFE_CAST(JSON_VALUE(attributes, '$.updated_at') AS TIMESTAMP))
        FROM `{project_id}.{dataset_id}.{intermediate_table_id}`
        """
        job = bq_client.query(query)
        result = job.result()

        last_updated_at_bq = None
        for row in result:
            last_updated_at_bq = row[0]

        if last_updated_at_bq:
            # BQ TIMESTAMP is UTC. Add buffer, then format to API's ISO format.
            effective_start_date = last_updated_at_bq + timedelta(seconds=1)
            effective_start_date_str = effective_start_date.isoformat(timespec='milliseconds')
            print(f"Incremental update: Fetching data updated after {effective_start_date_str} (based on BigQuery, ISO formatted).")
        else:
            # Table is empty or no updated_at found, fetch all data from a very old date (UTC)
            effective_start_date_str = '2000-01-01T00:00:00.000Z'
            print(f"No existing data in BigQuery. Fetching all data from {effective_start_date_str} (UTC).")
    except Exception as e:
        print(f"Error querying BigQuery for last updated_at: {e}. Defaulting to full reload from '2000-01-01'.")
        effective_start_date_str = '2000-01-01T00:00:00.000Z'

# Determine the last processed page for resuming by checking existing temporary tables
# Corrected query to use REGEXP_EXTRACT and CAST for parsing page numbers
query = f"SELECT CAST(REGEXP_EXTRACT(table_id, r'{temp_table_prefix}(\d+)') AS INT64) as page_num FROM `{project_id}.{dataset_id}.__TABLES__` WHERE table_id LIKE '{temp_table_prefix}%'"

last_processed_page = 0
# Removed try-except block to allow error to propagate if query fails
existing_pages_df = bq_client.query(query).to_dataframe()
if not existing_pages_df.empty:
    last_processed_page = existing_pages_df['page_num'].max()


page_num = last_processed_page + 1
print(f"Starting data retrieval from page {page_num}.")

max_retries = 3
retry_delay = 5 # seconds

# Remove the Cookie header as it's not necessary according to API documentation
if 'Cookie' in headers:
    del headers['Cookie']

total_records_retrieved = 0
pages_processed_this_run = 0

# Add filtering parameters if an effective_start_date_str was determined
if effective_start_date_str:
    params['filter_by'] = 'updated_at' # Changed from 'attributes.updated_at'
    params['filter_with'] = '>='
    params['filter_on'] = effective_start_date_str

if END_DATE_STR:
    print("\n--- IMPORTANT NOTE ON END DATE FILTERING ---")
    print("Warning: The Lawmatics API currently supports only a single date filter per request.")
    print("The 'END_DATE_STR' you provided will *not* be applied as an API filter during data retrieval.")
    print("Data will be retrieved from the 'START_DATE_STR' (or last updated date) onwards.")
    print("If you need to limit results by an end date, please apply additional filtering in BigQuery after retrieval.")
    print("-------------------------------------------\n")

while True:
    # Check if we've hit our page limit
    if MAX_PAGES_TO_PROCESS is not None and pages_processed_this_run >= MAX_PAGES_TO_PROCESS:
        print(f"Reached maximum pages limit ({MAX_PAGES_TO_PROCESS}). Stopping.")
        break

    params['page'] = page_num
    params['sort_order'] = 'asc'
    params['sort_by'] = 'updated_at'
    print(f"Retrieving page {page_num} with filters: {params.get('filter_by')}, {params.get('filter_with')}, {params.get('filter_on')}")

    retries = 0
    while retries < max_retries:
        if retries > 0: print(f"Retry number {retries}")
        try:
            # Allow redirects by default (requests handles --location)
            response = requests.get(api_endpoint, headers=headers, params=params)
            response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
            page_data = response.json()

            if 'data' in page_data and isinstance(page_data['data'], list):
                current_page_items = page_data['data']
                if not current_page_items:  # Check if the 'data' list is empty
                    print("Empty data list received, assuming end of data.")
                    break # No more data

                # Convert to DataFrame
                df_page = pd.DataFrame(current_page_items)

                # Convert attributes column to JSON string to avoid schema conflicts
                if 'attributes' in df_page.columns:
                    df_page['attributes'] = df_page['attributes'].apply(lambda x: pd.Series(x).to_json() if pd.notna(x) else None)

                # Define a consistent schema to prevent BigQuery from inferring different types
                table_schema = [
                    bigquery.SchemaField("id", "STRING"),
                    bigquery.SchemaField("type", "STRING"),
                    bigquery.SchemaField("attributes", "STRING"),  # Force this to be STRING
                    bigquery.SchemaField("relationships", "RECORD", fields=[
                        bigquery.SchemaField("owner", "RECORD", fields=[
                            bigquery.SchemaField("data", "RECORD", fields=[
                                bigquery.SchemaField("id", "STRING"),
                                bigquery.SchemaField("type", "STRING")
                            ])
                        ]),
                        bigquery.SchemaField("recipient", "RECORD", fields=[
                            bigquery.SchemaField("data", "RECORD", fields=[
                                bigquery.SchemaField("id", "STRING"),
                                bigquery.SchemaField("type", "STRING")
                            ])
                        ]),
                        bigquery.SchemaField("tracked", "RECORD", fields=[
                            bigquery.SchemaField("data", "RECORD", fields=[
                                bigquery.SchemaField("id", "STRING"),
                                bigquery.SchemaField("type", "STRING")
                            ])
                        ])
                    ])
                ]

                temp_table_id = f"{dataset_id}.{temp_table_prefix}{page_num}"

                pandas_gbq.to_gbq(
                    df_page,
                    temp_table_id,
                    project_id=project_id,
                    if_exists='replace' # Overwrite if table for this page already exists
                )
                print(f"Saved page {page_num} to BigQuery table {temp_table_id} with {len(current_page_items)} records.")
                total_records_retrieved += len(current_page_items)

                page_num += 1
                pages_processed_this_run += 1
                time.sleep(1) # Add a small delay to avoid overwhelming the API
                break # Break out of retry loop on success
            else:
                print("API response does not contain expected 'data' key with a list.")
                print(f"Response content: {page_data}")
                break # Exit if response structure is unexpected

        except requests.exceptions.RequestException as e:
            retries += 1
            print(f"Error during API request (Attempt {retries}/{max_retries}): {e}. Retrying...")
            if retries < max_retries:
                time.sleep(retry_delay)
            else:
                print("Max retries reached. Exiting.")
                break # Exit retry loop after max retries

    # Check if we broke out of the inner retry loop due to max retries or no data
    if retries == max_retries or (page_data and 'data' in page_data and not page_data['data']):
        break # Exit main loop if max retries reached or no data received in the last successful attempt

print(f"Retrieved and saved {total_records_retrieved} total records across {pages_processed_this_run} pages into temporary BigQuery tables.")
if MAX_PAGES_TO_PROCESS is not None:
    print(f"Processing was limited to {MAX_PAGES_TO_PROCESS} pages for development purposes.")

<>:121: SyntaxWarning: invalid escape sequence '\d'
<>:121: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-4219185473.py:121: SyntaxWarning: invalid escape sequence '\d'
  query = f"SELECT CAST(REGEXP_EXTRACT(table_id, r'{temp_table_prefix}(\d+)') AS INT64) as page_num FROM `{project_id}.{dataset_id}.__TABLES__` WHERE table_id LIKE '{temp_table_prefix}%'"


Secret retrieved successfully: (sha1) c32003bafa4577ecdbce1036cc7cedc098ae378a
Using user-defined start date: 2025-10-21, formatted as 2025-10-21T00:00:00.000+00:00 for API.
Starting data retrieval from page 1.
Retrieving page 1 with filters: updated_at, >=, 2025-10-21T00:00:00.000+00:00


100%|██████████| 1/1 [00:00<00:00, 9320.68it/s]


Saved page 1 to BigQuery table mlo_kpi.tmp_lawmatics_activity_page_1 with 25 records.
Retrieving page 2 with filters: updated_at, >=, 2025-10-21T00:00:00.000+00:00


100%|██████████| 1/1 [00:00<00:00, 5511.57it/s]


...

Saved page 817 to BigQuery table mlo_kpi.tmp_lawmatics_activity_page_817 with 25 records.
Retrieving page 818 with filters: updated_at, >=, 2025-10-21T00:00:00.000+00:00


100%|██████████| 1/1 [00:00<00:00, 9137.92it/s]


Saved page 818 to BigQuery table mlo_kpi.tmp_lawmatics_activity_page_818 with 25 records.
Retrieving page 819 with filters: updated_at, >=, 2025-10-21T00:00:00.000+00:00


100%|██████████| 1/1 [00:00<00:00, 11715.93it/s]


Saved page 819 to BigQuery table mlo_kpi.tmp_lawmatics_activity_page_819 with 9 records.
Retrieving page 820 with filters: updated_at, >=, 2025-10-21T00:00:00.000+00:00
Empty data list received, assuming end of data.
Retrieved and saved 20459 total records across 819 pages into temporary BigQuery tables.


In [6]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound # Import NotFound exception

# BigQuery configuration
project_id = 'www-prod-389819'
dataset_id = 'mlo_kpi'
temp_table_prefix = 'tmp_lawmatics_activity_page_'
intermediate_table_full_id = f"{project_id}.{dataset_id}.lawmatics_activity_timeline_raw_retrieval" # Full ID for table reference

bq_client = bigquery.Client(project=project_id)

try:
    # 1. Check if the intermediate table exists and create it if not
    try:
        bq_client.get_table(intermediate_table_full_id) # API request
        print(f"Intermediate table {intermediate_table_full_id} already exists.")
    except NotFound:
        print(f"Intermediate table {intermediate_table_full_id} not found. Creating table.")
        table_schema_for_intermediate = [
            bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("attributes", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("owner", "RECORD", mode="NULLABLE", fields=[
                bigquery.SchemaField("data", "RECORD", mode="NULLABLE", fields=[
                    bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
                    bigquery.SchemaField("type", "STRING", mode="NULLABLE")
                ])
            ]),
            bigquery.SchemaField("recipient", "RECORD", mode="NULLABLE", fields=[
                bigquery.SchemaField("data", "RECORD", mode="NULLABLE", fields=[
                    bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
                    bigquery.SchemaField("type", "STRING", mode="NULLABLE")
                ])
            ]),
            bigquery.SchemaField("tracked", "RECORD", mode="NULLABLE", fields=[
                bigquery.SchemaField("data", "RECORD", mode="NULLABLE", fields=[
                    bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
                    bigquery.SchemaField("type", "STRING", mode="NULLABLE")
                ])
            ])
        ]
        table = bigquery.Table(intermediate_table_full_id, schema=table_schema_for_intermediate)
        bq_client.create_table(table) # API request
        print(f"Table {intermediate_table_full_id} created successfully.")

    # Get all temporary tables that exist to provide a count and for deletion later
    get_temp_tables_query = f"""
    SELECT table_id
    FROM `{project_id}.{dataset_id}.__TABLES__`
    WHERE table_id LIKE '{temp_table_prefix}%'
    """
    tables_to_process_df = bq_client.query(get_temp_tables_query).to_dataframe()

    if tables_to_process_df.empty:
        print("No temporary tables found to consolidate.")
    else:
        print(f"Found {len(tables_to_process_df)} temporary tables matching '{temp_table_prefix}*' to consolidate.")

        # 2. Consolidate data into an intermediate table using a MERGE statement
        # This ensures that existing records are updated and new records are inserted.
        merge_query = f"""
        MERGE INTO `{intermediate_table_full_id}` T
        USING (
            SELECT
                id,
                type,
                attributes,
                relationships.owner as owner,
                relationships.recipient as recipient,
                relationships.tracked as tracked
            FROM `{project_id}.{dataset_id}.{temp_table_prefix}*`
        ) S
        ON T.id = S.id
        WHEN MATCHED THEN
            UPDATE SET
                T.type = S.type,
                T.attributes = S.attributes,
                T.owner = S.owner,
                T.recipient = S.recipient,
                T.tracked = S.tracked
        WHEN NOT MATCHED THEN
            INSERT (id, type, attributes, owner, recipient, tracked)
            VALUES (S.id, S.type, S.attributes, S.owner, S.recipient, S.tracked);
        """
        print(f"Executing MERGE query to update/insert data into {intermediate_table_full_id} from temporary tables...")
        bq_client.query(merge_query).result() # .result() waits for job completion
        print(f"Successfully merged data from {len(tables_to_process_df)} temporary tables into {intermediate_table_full_id}.")

        # 3. Delete temporary tables
        print("\nDeleting temporary tables...")
        deleted_count = 0
        for table_id in tables_to_process_df['table_id']:
            try:
                table_ref = f"{project_id}.{dataset_id}.{table_id}"
                bq_client.delete_table(table_ref)
                print(f"Deleted: {table_id}")
                deleted_count += 1
            except Exception as e:
                print(f"Error deleting {table_id}: {e}")
        print(f"\nSuccessfully deleted {deleted_count} out of {len(tables_to_process_df)} temporary tables.")

except Exception as e:
    print(f"Error during consolidation or cleanup: {e}")

Intermediate table www-prod-389819.mlo_kpi.lawmatics_activity_timeline_raw_retrieval already exists.
Found 819 temporary tables matching 'tmp_lawmatics_activity_page_*' to consolidate.
Executing MERGE query to update/insert data into www-prod-389819.mlo_kpi.lawmatics_activity_timeline_raw_retrieval from temporary tables...
Error during consolidation or cleanup: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/www-prod-389819/queries/61ed89ef-34c5-4af6-a15e-9149f929c736?maxResults=0&location=US&prettyPrint=false: Value of type STRUCT<data STRING> cannot be assigned to T.tracked, which has type STRUCT<data STRUCT<id STRING, type STRING>> at [20:29]

Location: US
Job ID: 61ed89ef-34c5-4af6-a15e-9149f929c736

